## Final Project

In [2]:
# Import libraries

from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as sk
import os
import sys
from IPython.display import display, Image
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc
from six.moves import cPickle as pickle
from six.moves import range
from sklearn.preprocessing import OneHotEncoder
from sklearn import model_selection
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from timeit import default_timer as timer
# needed to see images
from IPython.display import display, Image

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

#import tensorflow as tf
import time
import pickle
import seaborn as sns
import librosa
current_palette = sns.color_palette()

import tarfile
t = tarfile.open('UrbanSound8K.tar.gz', mode="r:gz")
t.extractall()

In [3]:
raw_sound = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')
raw_sound.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [4]:
import soundfile as sf
data, samplerate = sf.read('UrbanSound8K/audio/fold1/102106-3-0-0.wav')
print(data, samplerate)

[[-0.01174927  0.03039551]
 [-0.01153564  0.02471924]
 [-0.01644897  0.01794434]
 ..., 
 [-0.00588989  0.00012207]
 [ 0.00314331  0.00585938]
 [ 0.00540161  0.00689697]] 44100


In [5]:
'''for i in range(1,11):
    fold_name = 'fold' + str(i)
    fold_list.append(fold_name)
print(fold_list)'''
fold_list = ['fold1', 'fold2', 'fold3', 'fold4', 'fold5', 'fold6', 'fold7', 'fold8', 'fold9', 'fold10']

['fold1', 'fold2', 'fold3', 'fold4', 'fold5', 'fold6', 'fold7', 'fold8', 'fold9', 'fold10']


In [6]:
from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wave
from scipy.fftpack import dct 

In [7]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz


In [11]:
start_time = time.time()
a,b,c,d,e = extract_feature('UrbanSound8K/audio/fold1/102106-3-0-0.wav')
end_time = time.time()
print('time to extract features from one file: {:.3f}sec'.format((end_time-start_time)/60))

time to extract features from one file: 0.004sec


C:\Anaconda3\lib\site-packages\numpy\core\numeric.py:482: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return array(a, dtype, copy=False, order=order)


In [12]:
print(a.shape,b.shape,c.shape,d.shape,e.shape)
print(a[0])
print(40+12+128+13)

(40,) (12,) (128,) (7,) (6,)
-253.201139054
193


In [13]:
from os import listdir
from os.path import isfile, join
mfcc_data = []
exception_count = 0

start_time = time.time()
for i in range(10):
    mypath = 'UrbanSound8K/audio/'+ fold_list[i] + '/'
    files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    for f in files:
        try:
            fn = mypath + f
            mfccs,chroma,mel,contrast,tonnetz = extract_feature(fn)
            features = np.empty((0,193))
            ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
            features = np.vstack([features,ext_features])
            
        except:
            print(f)
            exception_count += 1
            continue

        l_row = raw_sound.loc[raw_sound['slice_file_name']==f].values.tolist()
        label = l_row[0][-1]
        fold = i+1
    
        mfcc_data.append([features, features.shape, label, fold])
        
            #print(f,old_samplerate,ss)
        
print("Exceptions: ", exception_count)
end_time = time.time()
print(print("time taken: {0} minutes {1:.1f} seconds".format((end_time - start_time)//60, (end_time - start_time)%60)))

.DS_Store


C:\Anaconda3\lib\site-packages\numpy\core\numeric.py:482: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return array(a, dtype, copy=False, order=order)
C:\Anaconda3\lib\site-packages\librosa\core\pitch.py:148: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store
Exceptions:  10
time to extract features from all files: 77.450sec


In [6]:
mfcc_pd = pickle.load(open('feature_data.p',"rb"))

In [7]:
cols=["features", "shape","label", "fold"]
mfcc_pd = pd.DataFrame(data = mfcc_pd, columns=cols)

In [8]:
mfcc_pd.head()

,features,shape,label,fold
0,"[[-402.458131222, 92.2208198393, 19.3559458953...","(1, 193)",dog_bark,1
1,"[[-366.481581922, 126.422493153, 31.1635153803...","(1, 193)",dog_bark,1
2,"[[-421.545181629, 69.0278123077, 11.2546785074...","(1, 193)",dog_bark,1
3,"[[-253.201139054, 99.422307006, -42.0187279331...","(1, 193)",dog_bark,1
4,"[[-383.810140281, 128.451555696, 29.3132434931...","(1, 193)",gun_shot,1


In [9]:
labels = mfcc_pd['label']
classes = list(set(labels))
label_list = []

for i in range(10):
    label_list.append((i, classes[i]))

label_mapping = dict(label_list)
label_mapping

{0: 'dog_bark',
 1: 'air_conditioner',
 2: 'jackhammer',
 3: 'siren',
 4: 'engine_idling',
 5: 'street_music',
 6: 'car_horn',
 7: 'gun_shot',
 8: 'children_playing',
 9: 'drilling'}

In [10]:
# Convert label to class number
label_num = []
for label in mfcc_pd['label']:
    for i in range(10):
        if label == label_mapping[i]:
            label_num.append(i)
#mfcc_pd["label_num"] = label_num

In [11]:
# One-hot encoding the lable for one-versus-all classification
def one_hot(labels):
    """
    Convert a list of labels to one hot code ndarray.
    Parameter:
    labels: list of integer(0-9)
    """
    n = len(labels)
    m = 10
    label_array = np.zeros((n, m), dtype = np.int)
    for i in range(n):
        label_array[i,labels[i]] = 1   
    return label_array

In [12]:
#label_onehot = list(one_hot(mfcc_pd["label_num"]))
label_onehot = np.array(one_hot(label_num))
#mfcc_pd['label_onehot'] = label_onehot
for i in range(10):
    mfcc_pd[label_mapping[i][:8]] = label_onehot[:,i]

In [13]:
mfcc_pd.head()

,features,shape,label,fold,dog_bark,air_cond,jackhamm,siren,engine_i,street_m,car_horn,gun_shot,children,drilling
0,"[[-402.458131222, 92.2208198393, 19.3559458953...","(1, 193)",dog_bark,1,1,0,0,0,0,0,0,0,0,0
1,"[[-366.481581922, 126.422493153, 31.1635153803...","(1, 193)",dog_bark,1,1,0,0,0,0,0,0,0,0,0
2,"[[-421.545181629, 69.0278123077, 11.2546785074...","(1, 193)",dog_bark,1,1,0,0,0,0,0,0,0,0,0
3,"[[-253.201139054, 99.422307006, -42.0187279331...","(1, 193)",dog_bark,1,1,0,0,0,0,0,0,0,0,0
4,"[[-383.810140281, 128.451555696, 29.3132434931...","(1, 193)",gun_shot,1,0,0,0,0,0,0,0,1,0,0


In [14]:
ll = [mfcc_pd['features'][i].ravel() for i in range(mfcc_pd.shape[0])]
mfcc_pd['sample'] = pd.Series(ll, index=mfcc_pd.index)
del mfcc_pd['features']

In [15]:
mfcc_pd.head()

,shape,label,fold,dog_bark,air_cond,jackhamm,siren,engine_i,street_m,car_horn,gun_shot,children,drilling,sample
0,"(1, 193)",dog_bark,1,1,0,0,0,0,0,0,0,0,0,"[-402.458131222, 92.2208198393, 19.3559458953,..."
1,"(1, 193)",dog_bark,1,1,0,0,0,0,0,0,0,0,0,"[-366.481581922, 126.422493153, 31.1635153803,..."
2,"(1, 193)",dog_bark,1,1,0,0,0,0,0,0,0,0,0,"[-421.545181629, 69.0278123077, 11.2546785074,..."
3,"(1, 193)",dog_bark,1,1,0,0,0,0,0,0,0,0,0,"[-253.201139054, 99.422307006, -42.0187279331,..."
4,"(1, 193)",gun_shot,1,0,0,0,0,0,0,0,1,0,0,"[-383.810140281, 128.451555696, 29.3132434931,..."


In [16]:
type(mfcc_pd['sample'])

pandas.core.series.Series

In [24]:
pickle.dump(mfcc_pd,open('193_features.p','wb'))

In [17]:
#http://www.practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/
#http://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html

In [18]:
# Random Forest classifier. Use fold10 as test set. Use fold1-9 for 9-fold cross validation
forest = OneVsRestClassifier(RandomForestClassifier(n_estimators = 500, max_depth=20, min_samples_leaf=30))

In [19]:
train_data = list(mfcc_pd.loc[mfcc_pd['fold'] != 1]['sample'])

In [20]:
train_data = np.array(train_data)
train_data.shape

(7858, 193)

In [22]:
class_list = [value for value in label_mapping.values()]
class_list

['dog_bark',
 'air_conditioner',
 'jackhammer',
 'siren',
 'engine_idling',
 'street_music',
 'car_horn',
 'gun_shot',
 'children_playing',
 'drilling']

In [25]:
# save fold 10 as test_data and perform a 9-fold cross validation
from sklearn.metrics import roc_auc_score
for i in range(1,10):
    train_data = np.array(list(mfcc_pd.loc[mfcc_pd['fold'] != i]['sample']))
    train_label = np.array(mfcc_pd.loc[mfcc_pd['fold'] != i]['dog_bark'])
    validation_data = np.array(list(mfcc_pd.loc[mfcc_pd['fold'] == i]['sample']))
    validation_label = np.array(mfcc_pd.loc[mfcc_pd['fold'] == i]['dog_bark'])
    roc = []
    
    forestmodel = forest.fit(train_data, train_label)
    train_accuracy = forestmodel.score(train_data,train_label)
    validation_accuracy = forestmodel.score(validation_data,validation_label)
    accuracy.append(validation_accuracy) 
    print(i)
    print("train accuracy: ", train_accuracy)
    print("validation accuracy: ", validation_accuracy)
    
print(accuracy)
    

NameError: name 'accuracy' is not defined

In [ ]:
print(set(mfcc_pd['fold']))
ss = np.array(validation_label)

In [ ]:
type(ss[0])